In [1]:
import pandas as pd

In [2]:
import os
import re
import string
import math
from decimal import Decimal
import nltk

In [8]:
import xlrd

In [9]:
from nltk.stem.isri import ISRIStemmer

In [12]:
!pip install openpyxl

  Using cached https://files.pythonhosted.org/packages/96/c2/3dd434b0108730014f1b96fd286040dc3bcb70066346f7e01ec2ac95865f/et_xmlfile-1.1.0-py3-none-any.whl


In [13]:
df = pd.read_excel("arabic_data_6000.xlsx",engine='openpyxl')

In [14]:
df.head()

,Sentences,Class
0,لدي اقتراح على مجلس النواب القادم بالتصويت على...,0
1,ما هي الخدمة التي تقدمها الحكومة مقابل استيفاء...,0
2,فرض ضريبة من غير مقابل هو دليل على الإفلاس الف...,0
3,ما ان نسير الى الامام حتى نجد من يرغب في اعاتن...,0
4,تعددت المسائل المالية التي تم تعديلها من قبل ا...,0


In [65]:
df.Class.nunique()

3

In [15]:
df["Sentences"][0]

'لدي اقتراح على مجلس النواب القادم بالتصويت على قانون يحدد مدة عمل الحكومة والوزراء لمدة زمنية واضحة لا تقل عن ثلاث سنوات وكذلك تثبيت جميع القوانين المؤقتة والدائمة لمدة زمنية لا تقل عن خمس سنوات وذلك لكي يتسنى للمواطن وللمستثمر وللسائح وللزائر ان يضع خطة لحياته او لعمله او لزيارته ولا يتفاجأ بتغيير القوانين كل يوم وكأننا نعيش في بورصة قوانين وبورصة وزراء , ارحمونا واثبتوا على رأي واحد لمدة سنة على الاقل لم يعد بوسعنا تحمل هذا التخبط الدائم'

In [16]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6000 entries, 0 to 5999
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Sentences  5983 non-null   object
 1   Class      6000 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 93.9+ KB


In [17]:
df.dropna(inplace=True)

In [18]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5983 entries, 0 to 5999
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Sentences  5983 non-null   object
 1   Class      5983 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 140.2+ KB


In [19]:
df.reset_index(drop=True,inplace=True)

In [33]:
X = df["Sentences"]
y = df["Class"]

In [36]:
from sklearn.feature_extraction.text import CountVectorizer , TfidfTransformer
from sklearn.model_selection import train_test_split



In [35]:
vectorizer = CountVectorizer()

In [37]:
X_counts = vectorizer.fit_transform(X)

In [39]:
X_counts.shape

(5983, 37373)

In [40]:
tfidf = TfidfTransformer()

In [50]:
X_train, X_test, y_train, y_test = train_test_split(X_counts, y, test_size = 0.15, random_state = 0)

In [51]:
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(5085, 37373)
(5085,)
(898, 37373)
(898,)


In [52]:
X_train_tfidf = tfidf.fit_transform(X_train)
X_train_tfidf.toarray()
X_test_tfidf = tfidf.fit_transform(X_test)
X_test_tfidf.toarray()

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [55]:
!pip install imblearn

In [57]:
from imblearn.over_sampling import RandomOverSampler
sampler=RandomOverSampler()
X_train, y_train = sampler.fit_resample(X_train_tfidf, y_train)
X_test, y_test = sampler.fit_resample(X_test_tfidf, y_test)

In [60]:
from sklearn.svm import SVC

clf= SVC(kernel = 'rbf', random_state = 0)
clf.fit(X_train, y_train)
clf.score(X_train, y_train)

0.9998038061604866

In [61]:
y_pred=clf.predict(X_test)

In [64]:
from sklearn.metrics import confusion_matrix,accuracy_score,classification_report
cm = confusion_matrix(y_test, y_pred)
Accuracy_Score = accuracy_score(y_test, y_pred)
print(cm)
print(Accuracy_Score)
print(classification_report(y_test,y_pred))

[[281  28   2]
 [ 33 275   3]
 [ 16  16 279]]
0.894962486602358
              precision    recall  f1-score   support

           0       0.85      0.90      0.88       311
           1       0.86      0.88      0.87       311
           2       0.98      0.90      0.94       311

    accuracy                           0.89       933
   macro avg       0.90      0.89      0.90       933
weighted avg       0.90      0.89      0.90       933

